In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nutrition-health-survey-age-prediction/Train_Data.csv
/kaggle/input/nutrition-health-survey-age-prediction/Test_Data.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the data
train_data = pd.read_csv('/kaggle/input/nutrition-health-survey-age-prediction/Train_Data.csv')
test_data = pd.read_csv('/kaggle/input/nutrition-health-survey-age-prediction/Test_Data.csv')

In [3]:
import warnings

warnings.filterwarnings('ignore')

print(train_data.head())
print(train_data.info())
print(train_data.isnull().sum())

print(test_data.head())
print(test_data.info())
print(test_data.isnull().sum())

      SEQN  RIAGENDR  PAQ605  BMXBMI  LBXGLU  DIQ010  LBXGLT  LBXIN age_group
0  73564.0       2.0     2.0    35.7   110.0     2.0   150.0  14.91     Adult
1  73568.0       2.0     2.0    20.3    89.0     2.0    80.0   3.85     Adult
2  73576.0       1.0     2.0    23.2    89.0     2.0    68.0   6.14     Adult
3  73577.0       1.0     2.0    28.9   104.0     NaN    84.0  16.15     Adult
4  73580.0       2.0     1.0    35.9   103.0     2.0    81.0  10.92     Adult
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1966 entries, 0 to 1965
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   SEQN       1954 non-null   float64
 1   RIAGENDR   1948 non-null   float64
 2   PAQ605     1953 non-null   float64
 3   BMXBMI     1948 non-null   float64
 4   LBXGLU     1953 non-null   float64
 5   DIQ010     1948 non-null   float64
 6   LBXGLT     1955 non-null   float64
 7   LBXIN      1957 non-null   float64
 8   age_group  1952 non-n

In [4]:
# Identify numeric columns
numeric_cols = train_data.select_dtypes(include=[np.number]).columns
non_numeric_cols = train_data.select_dtypes(exclude=[np.number]).columns

In [5]:
# Fill NaN values with the mean of each numeric column
train_data[numeric_cols].fillna(train_data[numeric_cols].mean(), inplace=True)
test_data[numeric_cols].fillna(test_data[numeric_cols].mean(), inplace=True)

In [6]:
# Fill NaN values for non-numeric columns with the mode
#for col in non_numeric_cols:
#    train_data[col].fillna(train_data[col].mode()[0], inplace=True)
#    test_data[col].fillna(test_data[col].mode()[0], inplace=True)

# For non-numeric columns, fill NaN values with the mode
for col in non_numeric_cols:
   if col in train_data.columns:
      train_data[col].fillna(train_data[col].mode()[0], inplace=True)

# Check if column exists in the test data before filling
if col in test_data.columns:
    test_data[col].fillna(test_data[col].mode()[0], inplace=True)

In [7]:


# Separate features and target variable
X = train_data.drop(columns=['age_group', 'SEQN'])
y = train_data['age_group']

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(test_data.drop(columns=['SEQN']))

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# Separate features and target
X = train_data.drop(columns=['age_group', 'SEQN'])
y = train_data['age_group']

X_test = test_data.drop(columns=['SEQN'])




In [11]:
# Identify numeric and categorical columns
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()


In [12]:
# Impute missing values for numeric columns with SimpleImputer(mean)
num_imputer = SimpleImputer(strategy='mean')

In [13]:
# If you have categorical columns, you can impute with mode (most frequent):
cat_imputer = SimpleImputer(strategy='most_frequent')


In [14]:
# Apply imputer on numeric columns
X_num = pd.DataFrame(num_imputer.fit_transform(X[numeric_cols]), columns=numeric_cols)
X_test_num = pd.DataFrame(num_imputer.transform(X_test[numeric_cols]), columns=numeric_cols)


In [15]:
# Apply imputer on categorical columns (if exist)
if categorical_cols:
   X_cat = pd.DataFrame(cat_imputer.fit_transform(X[categorical_cols]), columns=categorical_cols)
   X_test_cat = pd.DataFrame(cat_imputer.transform(X_test[categorical_cols]), columns=categorical_cols)

# Combine numeric + categorical columns back
   X = pd.concat([X_num, X_cat], axis=1)
   X_test = pd.concat([X_test_num, X_test_cat], axis=1)
else:
    X = X_num
    X_test = X_test_num

In [16]:
# Now split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:
# Scale features (only numeric features, but since after imputation and concat we have all numeric)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)


In [18]:
# Train RandomForestClassifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=42)

In [19]:
# Validate model
y_pred_val = model.predict(X_val_scaled)
accuracy = accuracy_score(y_val, y_pred_val)
print(f'Validation accuracy: {accuracy:.4f}')


Validation accuracy: 0.8426


In [20]:
# Predict on test set
y_pred_test = model.predict(X_test_scaled)




In [21]:
# Create submission file (remember age_group: Adult = 0, Senior = 1)
submission = pd.DataFrame({'SEQN': test_data['SEQN'], 'age_group': y_pred_test})
submission.to_csv('sample-submission.csv', index=False)